In [1]:
!pip install statsmodels

In [2]:
import pandas as pd
from collections import Counter
from statsmodels.stats.inter_rater import fleiss_kappa

In [3]:
df = pd.read_csv('../../data/judged/LLM-annotation-results-human_choice.csv')

In [4]:
df.shape

(568, 4)

In [5]:
df.head(2)

,timestamp,evaluation_id,human_choice,human_id
0,30/09/2025 06:59:16,CG019_3_General Knowledge_minimum_gemini-1.5-p...,B,user-1759226290086-spowi1b0l
1,30/09/2025 06:59:31,CG006_4_General Knowledge_contextual_gemini-1....,A,user-1759226290086-spowi1b0l


In [6]:
print('Total de Anotações')
len(df)

Total de Anotações


568

In [7]:
num_anotadores = df['human_id'].nunique()
num_pares_avaliados = df['evaluation_id'].nunique()
print(df['human_id'].value_counts().head())
print(f"\nNúmero de anotadores únicos: {num_anotadores}")
print(f"Número de pares únicos avaliados: {num_pares_avaliados}")
print(f"Média de anotações por pessoa: {df['human_id'].value_counts().mean():.2f}")

human_id
user-1759172933100-gj9oyyc0l    93
user-1759257213865-94o4sjndz    66
user-1759166974830-mu6fsc7yf    46
user-1759256521302-5inns438p    18
user-1759273239005-wsyiev3t2    16
Name: count, dtype: int64

Número de anotadores únicos: 68
Número de pares únicos avaliados: 150
Média de anotações por pessoa: 8.35


In [7]:
contagem_por_par = df['evaluation_id'].value_counts()
print(contagem_por_par.head())
print("\nEstatísticas da contagem por par:")
print(f"Média de anotações: {contagem_por_par.mean():.2f}")
print(f"Mínimo de anotações: {contagem_por_par.min()}")
print(f"Máximo de anotações: {contagem_por_par.max()}")

evaluation_id
CR019_4_Creative_gpt-4o_contextual_vs_structured_pt                   10
CR020_4_Creative_detailed_gemini-1.5-pro-latest_vs_gpt-4o_pt           9
CR005_3_Creative_gpt-4o_minimum_vs_structured_pt                       9
CR015_3_Creative_gemini-1.5-pro-latest_contextual_vs_structured_pt     8
CR015_1_Creative_contextual_gpt-4o_vs_llama-3.3-70b-versatile_pt       8
Name: count, dtype: int64

Estatísticas da contagem por par:
Média de anotações: 3.79
Mínimo de anotações: 2
Máximo de anotações: 10


In [8]:
print("Distribuição das Escolhas Humanas (A, B, Empate)")
print(df['human_choice'].value_counts(normalize=True).apply('{:.2%}'.format))

Distribuição das Escolhas Humanas (A, B, Empate)
human_choice
B         45.60%
A         38.73%
Empate    15.67%
Name: proportion, dtype: object


## Geração do Ground Truth

In [9]:
def calcular_vencedor_maioria(votos):
    """
    Calcula o voto da maioria para uma série de votos.
    - Se houver um vencedor claro por maioria ('A' ou 'B'), retorna esse vencedor.
    - Em todos os outros casos (empate entre A e B, ou votos espalhados), retorna 'Tie'.
    """
    contagem = Counter(votos)
    votos_para_a = contagem.get('A', 0)
    votos_para_b = contagem.get('B', 0)
    
    if votos_para_a > votos_para_b:
        return 'A'
    elif votos_para_b > votos_para_a:
        return 'B'
    else:
        return 'Tie'

In [10]:
# Agrupar por 'evaluation_id' e a calcular o vencedor por maioria

# Agrupa por evaluation_id e aplica a função de voto de maioria
ground_truth = df.groupby('evaluation_id')['human_choice'].apply(calcular_vencedor_maioria).reset_index()
ground_truth.rename(columns={'human_choice': 'human_winner'}, inplace=True)

In [11]:
ground_truth.head()

,evaluation_id,human_winner
0,CG001_sabia-3.1_2_General Knowledge_detailed_e...,Tie
1,CG002_1_General Knowledge_minimum_gemini-1.5-p...,B
2,CG002_2_General Knowledge_sabia-3.1_contextual...,B
3,CG002_3_General Knowledge_sabia-3.1_detailed_v...,A
4,CG002_5_General Knowledge_minimum_gemini-1.5-p...,Tie


In [12]:
print("Distribuição das Escolhas Humanas (A, B, Empate) no ground_truth")
print(ground_truth['human_winner'].value_counts(normalize=True).apply('{:.2%}'.format))

Distribuição das Escolhas Humanas (A, B, Empate) no ground_truth
human_winner
B      46.00%
A      38.00%
Tie    16.00%
Name: proportion, dtype: object


In [13]:
ground_truth.to_csv('../../data/judged/human_ground_truth.csv', index=False, encoding='utf-8-sig')

In [14]:
len(ground_truth)

150

## A calcular a concordância entre os avaliadores (Fleiss' Kappa)

In [15]:
contagem_por_par = df['evaluation_id'].value_counts()
min_votos = contagem_por_par.min()
print(f"O número mínimo de votos por par é {min_votos}. Vamos usar este valor para a amostragem.")

O número mínimo de votos por par é 2. Vamos usar este valor para a amostragem.


In [16]:
# Agrupa por 'evaluation_id' e pega numa amostra aleatória de 'min_votos' de cada grupo.
df_amostra_kappa = df.groupby('evaluation_id').sample(n=min_votos, random_state=42)

In [17]:
# preparar os dados da amostra para o formato que o Fleiss' Kappa espera
# ele precisa de uma tabela onde as linhas são os pares (subjects)
# e as colunas são as categorias de escolha (A, B, Empate).
counts = df_amostra_kappa.groupby(['evaluation_id', 'human_choice']).size().unstack(fill_value=0)
counts = counts[['A', 'B', 'Empate']]
counts.head(2)

human_choice,A,B,Empate
evaluation_id,,,
CG001_sabia-3.1_2_General Knowledge_detailed_en_vs_pt,0,1,1
CG002_1_General Knowledge_minimum_gemini-1.5-pro-latest_vs_llama-3.3-70b-versatile_pt,0,0,2


In [18]:
kappa_value = fleiss_kappa(counts.values)
print(f"\nO valor do Fleiss' Kappa é: {kappa_value:.4f}")


O valor do Fleiss' Kappa é: 0.1780


In [19]:
if kappa_value > 0.8:
    print("Interpretação: Concordância quase perfeita. Os seus dados são muito fiáveis!")
elif kappa_value > 0.6:
    print("Interpretação: Concordância substancial. Pode usar os dados com confiança.")
elif kappa_value > 0.4:
    print("Interpretação: Concordância moderada. Os dados são utilizáveis.")
else:
    print("Interpretação: Concordância fraca. A fiabilidade dos dados é baixa.")

Interpretação: Concordância fraca. A fiabilidade dos dados é baixa.
